In [4]:
import joblib
import numpy as np
import pandas as pd
import datetime, random

In [5]:
df = joblib.load('../DataBase.pkl').iloc[0:10]
df

,ImagePath,Class,Caption,Age,ImageFile,PatientName,Sex,BirthDate
0,선천성유문협착증,1,This plain abdominal supine radiograph shows m...,6,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,윤경경,F,20190302
1,선천성유문협착증,1,This supine plain abdominal radiograph shows m...,5,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,박재진,F,20200919
2,선천성유문협착증,1,This plain abdominal supine radiograph of an a...,6,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,정경도,M,20190212
3,선천성유문협착증,1,This plain supine abdominal radiograph of an a...,3,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,강호경,M,20220805
4,선천성유문협착증,1,This plain supine abdominal radiograph shows m...,7,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,김선수,M,20181024
5,선천성유문협착증,1,This supine plain abdominal radiograph reveals...,4,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,윤영우,F,20210401
6,선천성유문협착증,1,This plain abdominal supine radiograph of an a...,6,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,이준현,M,20190110
7,선천성유문협착증,1,This plain abdominal supine radiograph reveals...,6,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,최경진,F,20190124
8,선천성유문협착증,1,This plain supine abdominal radiograph shows m...,4,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,임윤수,F,20211019
9,선천성유문협착증,1,This plain abdominal supine radiograph shows a...,5,../Data/Validation/01.원천데이터/VS_1.질환_1.선천성유문협착증...,장서성,M,20200914


In [6]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import pydicom
from pydicom.dataset import FileDataset
from datetime import datetime

# DICOM 저장 폴더 생성
os.makedirs("output_dicom", exist_ok=True)

for idx, row in df.iterrows():
    # 이미지 불러오기 (RGB 변환)
    img = Image.open(row["ImageFile"]).convert("RGB")
    pixel_array = np.array(img)

    # DICOM 메타데이터 생성
    file_meta = pydicom.Dataset()
    file_meta.MediaStorageSOPClassUID = pydicom.uid.SecondaryCaptureImageStorage
    file_meta.MediaStorageSOPInstanceUID = pydicom.uid.generate_uid()
    file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian

    # FileDataset 생성
    ds = FileDataset(None, {}, file_meta=file_meta, preamble=b"\0" * 128)

    # 한글 지원 (UTF-8)
    ds.SpecificCharacterSet = "ISO_IR 192"

    # 환자 정보
    ds.PatientName = row["PatientName"]
    ds.PatientID = str(idx+1)  # ID는 고유하게 index 사용
    ds.PatientSex = row["Sex"]  # M/F
    ds.PatientAge = f"{int(row['Age']):03d}Y"  # DICOM 규격: 3자리+Y (예: "006Y")
    ds.PatientBirthDate = row["BirthDate"]
    
    # Study / Series / SOP UID
    ds.StudyInstanceUID = pydicom.uid.generate_uid()
    ds.SeriesInstanceUID = pydicom.uid.generate_uid()
    ds.SOPInstanceUID = file_meta.MediaStorageSOPInstanceUID
    ds.SOPClassUID = file_meta.MediaStorageSOPClassUID
    ds.StudyDescription = row["ImagePath"]

    # 날짜/시간
    dt = datetime.now()
    ds.StudyDate = dt.strftime("%Y%m%d")
    ds.StudyTime = dt.strftime("%H%M%S")
    ds.Modality = "OT"  # Other

    # 이미지 데이터
    ds.Rows, ds.Columns, _ = pixel_array.shape
    ds.SamplesPerPixel = 3
    ds.PhotometricInterpretation = "RGB"
    ds.PlanarConfiguration = 0
    ds.BitsAllocated = 8
    ds.BitsStored = 8
    ds.HighBit = 7
    ds.PixelRepresentation = 0
    ds.PixelData = pixel_array.tobytes()

    # 저장
    save_path = os.path.join("output_dicom", f"{idx}.dcm")
    ds.save_as(save_path)
    print(f"[{idx}] DICOM 저장 완료 → {save_path}")


[0] DICOM 저장 완료 → output_dicom\0.dcm
[1] DICOM 저장 완료 → output_dicom\1.dcm
[2] DICOM 저장 완료 → output_dicom\2.dcm
[3] DICOM 저장 완료 → output_dicom\3.dcm
[4] DICOM 저장 완료 → output_dicom\4.dcm
[5] DICOM 저장 완료 → output_dicom\5.dcm
[6] DICOM 저장 완료 → output_dicom\6.dcm
[7] DICOM 저장 완료 → output_dicom\7.dcm
[8] DICOM 저장 완료 → output_dicom\8.dcm
[9] DICOM 저장 완료 → output_dicom\9.dcm
